In [1]:
# important for gpuhub
# !pip install -r ../../requirements.txt --upgrade

## Importing Libraries and tokens

In [2]:
import wandb
import os
import torch

# load .env file
from dotenv import load_dotenv
from wandb_downloader import WandbDownloader
from geo_model_tester import GeoModelTester
from image_data_handler_test import TestImageDataHandler

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
WANDB_TOKEN = os.getenv('WANDB_TOKEN')
# Define where to run
env_path = '../../.env'
if not WANDB_TOKEN and os.path.exists(env_path):
  load_dotenv(env_path)
  WANDB_TOKEN = os.getenv('WANDB_TOKEN')

In [4]:
# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available.")
    
    # Print the name of the GPU
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    
    # Print the total and available memory
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9  # Convert bytes to GB
    print(f"Total Memory: {total_memory:.2f} GB")

    allocated_memory = torch.cuda.memory_allocated(0) / 1e9  # Convert bytes to GB
    print(f"Allocated Memory: {allocated_memory:.2f} GB")

    cached_memory = torch.cuda.memory_reserved(0) / 1e9  # Convert bytes to GB
    print(f"Cached Memory: {cached_memory:.2f} GB")

    # Print other properties
    device_properties = torch.cuda.get_device_properties(0)
    print(f"CUDA Capability: {device_properties.major}.{device_properties.minor}")
    print(f"Multi-Processor Count: {device_properties.multi_processor_count}")
else:
    print("No GPU found.")

No GPU found.


## Loading files from wandb

In [5]:
wandb.login(key=WANDB_TOKEN) if WANDB_TOKEN else wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: luki-st (nlp_ls). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/lukasstoeckli/.netrc


True

In [6]:
# Example usage:
entity = "nlp_ls"
project = "dspro2-predicting-country"
metric_name = "Validation Accuracy Top 1"
data_augmentation = "full_augmentation_v2"  # Replace with the desired augmentation
datasize = 81505  # Replace with the desired datasize
file_names_to_download = [".pth", ".json"]
image_size = [80, 130]

downloader = WandbDownloader(entity, project, data_augmentation, datasize, image_size)
run_data = downloader.get_and_collect_best_runs(metric_name, file_names_to_download)

# Now run_data contains all necessary information for further processing
import pprint
pprint.pprint(run_data)

{'Best Run 1': {'files': {'artifact/858477759/wandb_manifest.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-country/10adb1sx/artifact/858477759/wandb_manifest.json',
                          'best_model': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-country/10adb1sx/best_model_checkpointmodel_efficientnet_b3_lr_0.01_opt_adamW_weightDecay_0.1_imgSize_[80, '
                                        '130]_predict_coordinates_False.pth',
                          'country_to_index.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-country/10adb1sx/country_to_index.json',
                          'files/country_to_index.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-country/10adb1sx/files/country_to_index.json',
                          'test_data': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-country/yns8ucfa/test_data.pth',
                          'wandb-metadata.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-country/10ad

In [7]:
# Print the validation accuracy for the top 1, 3, and 5 predictions
for i in [1, 3, 5]:
    print(f"Validation Accuracy Top {i}: ", run_data["Best Run 1"]["metrics"][f"Validation Accuracy Top {i}"])

Validation Accuracy Top 1:  0.38654070302435434
Validation Accuracy Top 3:  0.6087970063186308
Validation Accuracy Top 5:  0.7050487700141096


In [8]:
run_data["Best Run 1"]["files"]

{'artifact/858477759/wandb_manifest.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-country/10adb1sx/artifact/858477759/wandb_manifest.json',
 'best_model': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-country/10adb1sx/best_model_checkpointmodel_efficientnet_b3_lr_0.01_opt_adamW_weightDecay_0.1_imgSize_[80, 130]_predict_coordinates_False.pth',
 'country_to_index.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-country/10adb1sx/country_to_index.json',
 'files/country_to_index.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-country/10adb1sx/files/country_to_index.json',
 'wandb-metadata.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-country/10adb1sx/wandb-metadata.json',
 'wandb-summary.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-country/10adb1sx/wandb-summary.json',
 'test_data': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-country/yns8ucfa/test_data.pth'}

## Loading data and creating data loader

In [9]:
# Creating Dataloaders with the classes
test_dataset = run_data["Best Run 1"]["files"]['test_data']
files = run_data["Best Run 1"]["files"]
country_to_index = files.get('country_to_index.json', None)
region_to_index = files.get('region_to_index.json', None)
region_index_to_middle_point = files.get('region_index_to_middle_point.json', None)
region_index_to_country_index = files.get('region_index_to_country_index.json', None)

data_handler = TestImageDataHandler(test_dataset, country_to_index, region_to_index, region_index_to_middle_point, region_index_to_country_index)
test_dataloader = data_handler.test_loader
country_to_index = data_handler.country_to_index
region_to_index = data_handler.region_to_index
region_index_to_middle_point = data_handler.region_index_to_middle_point
region_index_to_country_index = data_handler.region_index_to_country_index

Loading test data from test_data.pth
Test data loaded.


## Evaluating the model

In [10]:
num_classes = run_data["Best Run 1"]["parameters"]["different_countries"]

geo_model_tester = GeoModelTester(
    datasize=1000, 
    train_dataloader=None, 
    val_dataloader=None, 
    test_dataloader=test_dataloader, 
    num_classes=num_classes, 
    predict_coordinates=False, 
    country_to_index=country_to_index,
    region_to_index=region_to_index,
    region_index_to_middle_point=region_index_to_middle_point,
    region_index_to_country_index=region_index_to_country_index,
    predict_regions=False
)

In [11]:
# TODO: Debug the region problem because old dataset does not have regions
# TODO: Test the model from best runs
# TODO: Show the different models with the best results (also do it for different data sizes and mapped/non-mapped data)
model_name = run_data["Best Run 1"]["parameters"]["model_name"]
pretrained_weights = run_data["Best Run 1"]["files"]['best_model']

geo_model_tester.test(model_type=model_name, model_path=pretrained_weights)

TypeError: 'bool' object is not subscriptable